In [25]:
import pandas as pd
import spacy
import glob
from gensim.models.word2vec import Word2Vec
import os,sys

In [2]:
nlp=spacy.load("fr")

In [33]:
odds=[(os.path.split(f)[1].replace(".txt",""),open(f).read()) for f in glob.glob("../data/ODD/*.txt")]


In [8]:
df = pd.read_json("../data/ucl_courses.json")
#concatenate interesting text features
df["text"]=df.content.astype(str)+"\n"+df.goal.astype(str)+"\n"+df.prerequisite.astype(str)+"\n"+df.theme

In [30]:
df["tokens"]=df.text.apply(lambda x : [w.text for w in nlp(x) if not w.is_stop and w.pos_ != "PUNCT" and w.pos_ !="SPACE"])

In [35]:
odds_tokens=[[w.text for w in nlp(odd) if not w.is_stop and w.pos_ != "PUNCT" and w.pos_ !="SPACE"] for name,odd  in odds]

In [36]:
model=Word2Vec.load("/home/denis/resources/w2vec/fr/fr.bin")
model.build_vocab(df.text.values.tolist()+odds,update=True)
model.train(df.text.values.tolist()+odds,total_examples=len(df.text.values.tolist()+odds),epochs=500,compute_loss=True)

(974297827, 4203161000)

In [40]:
for i,odd_token in enumerate(odds_tokens):
    df["odd_score_{}".format(odds[i][0])]=df.tokens.apply(lambda x:model.wv.wmdistance(odd_token,x))

In [43]:
df.sort_values(by="odd_score_14")[["class","odd_score_6","odd_score_12","odd_score_13","odd_score_14"]].head(30)

,class,odd_score_6,odd_score_12,odd_score_13,odd_score_14
761,Géopolitique des ressources naturelles,20.496437,18.698690,19.064376,18.494587
818,Food Microbiology,19.610320,19.126077,19.995535,18.873938
843,Cariologie et dentisterie conservatrice,19.536906,19.629643,20.192308,18.914449
814,Pollution de l'environnement,19.427147,19.255792,20.146853,18.914574
680,Projet intégré en diagnostic environnemental,20.390096,19.409191,20.086545,19.025155
3099,Distribution,20.275447,19.545504,20.830812,19.147005
605,Développement durable et transition,20.316868,18.627431,20.004766,19.195656
786,Hydrogeology and Geoenvironment,20.462693,20.185503,20.918871,19.224347
3111,Comptabilité et analyse des états financiers,20.925213,19.673740,20.735666,19.241359
784,Energies renouvelables,20.268270,19.071322,20.593626,19.261673


In [24]:
odds[1]

'Objectif 6 : Garantir l’accès de tous à des services d’alimentation en eau et d’assainissement gérés de façon durable\nUne eau propre et accessible pour tous est un élément essentiel du monde dans lequel nous voulons vivre. Il y a assez d’eau sur la planète pour réaliser ce rêve.\nMais du fait d’économies déficientes ou de mauvaises infrastructures, chaque année des millions de personnes, des enfants pour la plupart, meurent de maladies liées à l’insuffisance de leur approvisionnement en eau et à un manque d’installations sanitaires et d’hygiène.\n\nLes pénuries d’eau ou la mauvaise qualité de celle-ci et le manque de sanitaires ont un impact négatif sur la sécurité alimentaire, sur les choix de vie et sur les chances en matière d’éducation pour les familles pauvres à travers le monde. À l’heure actuelle, plus de 2 milliards de personnes risquent d’avoir un accès réduit aux ressources en eau douce et d’ici à 2050, au moins une personne sur quatre vivra probablement dans un pays touché